# Assignment 1

<div class="alert alert-success">This notebook provides a step by step guide to execute the code for Assignment 1 of cloud computing, which has two part.</div>

## <U>PART 1:</U>

### Setup : Readying environment
The first step is to import all the needed libraries that will be used throughout the notebook:

In [ ]:
import hashlib
import time
import requests
import pandas as pd

### Setup: Access to Marvel's API

The below step connects to marvel's API (https://developer.marvel.com/) which requires the following:

<ul>
  <li>Public key</li>
  <li>Private key</li>
  <li>Timestamp</li>
  <li>An md5 hash</li>
</ul>

In [ ]:
public_key = '46180a82e99e236b44c9ebb179e70221'
private_key = 'Insert your private key'
timestamp_str = str(time.time())
hash_1 = timestamp_str+private_key+public_key
hash_2 = hashlib.md5(hash_1.encode())
hashkey = hash_2.hexdigest()

### Step 1: Generate a list of 30 characters

The below code sends a request to Marvel's API to generate a list of 30 characters. To make this request the url with the 'characters' endpoint is used. The required parameters setup above are inserted below to gain authorization.

In [ ]:
urlCharacters = 'http://gateway.marvel.com/v1/public/characters'
params = {'apikey': public_key,
         'ts': timestamp_str,
         'hash': hashkey,
         'limit': 30}
response = requests.get(urlCharacters, params)
response = response.json()

name_list = []
for obj in response['data']['results']:
    name_list.append(obj['name'])

print(name_list)

### Step 2: Retrieve IDs

The below code takes the above generated list of 30 characters and retrieves their IDs, and it saves it in the variable ```id_list``` as a string

In [ ]:
id_list = []
for obj in response['data']['results']:
    id_list.append(str(obj['id']))

print(id_list)

### Step 3: Retrieve Total Events

The total number of events available for all the 30 characters is generated below and saved in the variable ```totEvents``` as an integer.

In [ ]:
totEvents = {}
for obj in response['data']['results']:
    totEvents[str(obj['id'])] = obj['events']['available']
    
print(totEvents)

### Step 4: Retrieve Total Series

The total number of series available for all 30 characters is generated and saved in the variable ```totSeries``` as an integer.

In [ ]:
totSeries = {}
for obj in response['data']['results']:
    totSeries[str(obj['id'])] = obj['series']['available']
    
print(totSeries)

### Step 5: Retrieve Total Comics

The total number of comics available for all 30 characters is generated and saved in ```totComics``` as an integer.

In [ ]:
totComics = {}
for obj in response['data']['results']:
    totComics[str(obj['id'])] = obj['comics']['available']
    
print(totComics)

### Step 6: Retrieve the Price of the Most Expensive Comic

Retrieve the most expensive comic for each of the 30 characters in the list and its respective price. This is saved in ```maxPricePerCharacter``` in float form and USD.

In [ ]:
maxPricePerCharacter = {}

for obj in response['data']['results']:
    resp_comics = requests.get(obj['comics']['collectionURI'], params={'apikey': public_key,
                                                                       'ts': timestamp_str,
                                                                       'hash': hashkey,
                                                                       'characterId': obj['id']})
    resp_comics = resp_comics.json()
    max_price = 0
    for comic in resp_comics['data']['results']:
        if comic['prices'][0]['price'] > max_price:
            max_price = comic['prices'][0]['price']
    if max_price == 0:
        maxPricePerCharacter[obj['name']] = None
    else:
        maxPricePerCharacter[obj['name']] = float(max_price)

print(maxPricePerCharacter)

### Step 7: Store the Data in a pandas DataFrame
The data generated is stored in a pandas DataFrame ```df``` with the following columns: 
<ul>
  <li>Character Name</li>
  <li>Character ID</li>
  <li>Total Available Events</li>
  <li>Total Available Series</li>
  <li>Total Available Comics</li>
  <li>Price of the Most Expensive Comic</li>
</ul>

In case a Character is not featured in any Events, Series, or Comics the entry will be filled with "None"
In case a Character does not have a price the entry will be filled with "None"

In [ ]:
def replace_zeros(df, column_name):
    df[column_name].replace(to_replace = 0, value = None, inplace = True)

df = pd.DataFrame(columns = ['Character Name', 'Character ID', 'Total Available Events', 'Total Available Series', 'Total Available Comics', 'Price of the Most Expensive Comic'])
i = 0
while i < 30:
    entry = {'Character Name': name_list[i],
             'Character ID': id_list[i],
             'Total Available Events': totEvents[id_list[i]],
             'Total Available Series': totSeries[id_list[i]],
             'Total Available Comics': totComics[id_list[i]],
             'Price of the Most Expensive Comic': maxPricePerCharacter[name_list[i]]}
    df = df.append(entry, ignore_index = True)
    i += 1
    
replace_zeros(df, 'Total Available Events'), replace_zeros(df, 'Total Available Series'), replace_zeros(df, 'Total Available Comics')

df

### Step 8: Save the DataFrame to a CSV

The ```df``` file is saved as a CSV titled ```data.csv```

In [ ]:
df.to_csv("data.csv", index = False)

## <u> PART II: </u>

The second part creates an API that allows the user to interact with the DataFrame generated in the earlier step.

<b>Users will be able to do the following:</b>
<ul>
  <li>Retrieve the whole DataFrame in JSON format</li>
  <li>Retrieve a single entry or a list of entries using a Character Name or Character ID</li>
  <li>Add a new character to the DataFrame by specifying: Character Name, Character ID, Available Events, Available Series, Available Comics, Price of Comics</li>
  <li>Add a new character to the DataFrame by specifying only the Character ID, the API will extract the remaining data from Marvel's API and append it to the DataFrame.</li>
  <li>Delete a character or a list of characters by providing the Character ID or Character Name</li>
</ul>

<b>Users will not be able to do the following:</b>
<ul>
  <li>Add a new character with a pre-existing Character ID</li>
  <li>Delete a character that does not exist in the DataFrame</li>
  <li>Add a new character using Character ID that does not exist in Marvel's API</li>
</ul>

In order to add and delete characters authentication is required: users need to input an e-mail and a password, this authorization is limited to 1 hour.

Note: For some Mac OS versions, requests to http://localhost:5000/ will produce a 403 error. Try directing your requests to http://127.0.0.1:5000/ instead.

### Retrieve a single entry using Character ID

In [ ]:
resp = requests.get('http://localhost:5000/characters', params={'characterID': 1009148})
resp.json()

### Retrieve multiple entries using Character ID

In [ ]:
resp = requests.get('http://localhost:5000/characters', params={'characterID': [1009148, 1011031]})
resp.json()

### Attempt to add character without Authorization

In [ ]:
resp = requests.post('http://localhost:5000/characters', params={'characterID': 1017100,
                                                                'characterName': 'Marco',
                                                                'events': 4,
                                                                'series': 5,
                                                                'comics': 10,
                                                                'price': 0.0})
resp.json()

### Create Authorization

#### Signup Process

The below authorization process requires using the url endpoint 'signup', an e-mail, and a password.
Users will use this to "sign up" to use the API

In [ ]:
resp = requests.post('http://localhost:5000/signup', params={'email': 'safia@gmail.com',
                                                             'password': '1111'})
resp.json()

#### Login Process

In [ ]:
resp = requests.get('http://localhost:5000/login', params={'email': 'safia@gmail.com',
                                                             'password': '1111'})
resp.json()

Upon successfuly signing up, a token is generated. Please copy the token and paste it in the below ```token``` variable

In [ ]:
token = resp.json()['token']
token

### Delete Character with Authorization [Incorrect Character ID]

In [ ]:
resp = requests.delete('http://localhost:5000/characters', params={'characterID': 101710000}, headers={'Authorization': f'Bearer {token}'})
resp.json()

### Delete character with Authorization

In [ ]:
resp = requests.delete('http://localhost:5000/characters', params={'characterID': 1011334}, headers={'Authorization': f'Bearer {token}'})
resp.json()

# Bonus Question

The code below enables users to modify the Price of the Most Expensive Comic by providing either the Character ID or the Character Name. The API accept new prices in different currencies, including USD, EUR, GBP and CAD and transforms them to the right values to the exchange rate of the considered date and time. 

forex_python is used as it is a free foreign exhange rates and currency converter

In [ ]:
pip install forex_python 

In [ ]:
from forex_python.converter import CurrencyRates

The price of the most expensive comic can be modified by providing either the "characterName" or "characterID", the desired new "price" and the "currency" of the provided price. The result is always in USD. If no "currency" parameter is provided, it assumes the price to be in USD.

In [ ]:
requests.put('http://localhost:5000/characters', params={'characterID': 1016823, 'price':60, 'currency':'CAD'}).json()